# Scrape cronometer.com
I tracked my weight and everything I ate from 4/1/21 to 8/11/21. Selenium is used to log into the site and move from day to day. Pandas.read_html was used to scrape the data from the html tables. The data was used to create an article about my weight loss journey [link](https://blog.finxter.com/how-i-scraped-my-nutrition-data-from-cronometer-com/)


In [1]:
# import cronometer as crono
# from cronometer import By, EC, WebDriverWait, ActionChains
import time
import traceback
import re
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re

In [2]:
URL = 'https://cronometer.com/login/'
START_DATE = '2021-04-01'
END_DATE = '2021-08-11'

# testing
END_DATE = '2021-04-01'
DIARY_XPATH = '//span[contains(text(), "Diary")]'

In [3]:
def set_viewport_size(driver, width, height):
    window_size = driver.execute_script("""
        return [window.outerWidth - window.innerWidth + arguments[0],
          window.outerHeight - window.innerHeight + arguments[1]];
        """, width, height)
    driver.set_window_size(*window_size)

In [4]:
def click_button(driver, button_xpath):
    try:
        button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, button_xpath)))
    except Exception as e:
        print('error trying to click button', button_xpath)
        print(e)

    webdriver.ActionChains(driver).move_to_element(button).click(button).perform()

In [5]:
def log_in(driver):
    user_name = driver.find_element(By.NAME, 'username')
    password = driver.find_element(By.NAME, 'password')
    login = driver.find_element(By.ID, 'login-button')

    user_name.send_keys('OneHundredDaysPythonCB@gmail.com')
    password.send_keys('h9#dHQ+PL;102my')
    login.click()
    time.sleep(5)

    # doesn't help!
    # driver.execute_script("document.body.style.transform='scale(0.8)';")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # go to the diary page
    click_button(driver, DIARY_XPATH)

In [6]:
def get_driver(url):
    driver = webdriver.Firefox()
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(5)
    set_viewport_size(driver, 1920, 3200)
    log_in(driver)
    time.sleep(2)
    
    return driver

In [7]:
def goto_day(driver, target_date):
    last_year_xpath = "//div[contains(text(), '«')]"
    next_month_xpath = "//div[contains(text(), '›')]"
    last_month_xpath = "//div[contains(text(), '‹')]"

    target_date = datetime.strptime(target_date, '%Y-%m-%d')
    today = datetime.today()

    last_year_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, last_year_xpath)))
    next_month_button = driver.find_element(By.XPATH, next_month_xpath)
    last_month_button = driver.find_element(By.XPATH, last_month_xpath)

    for _ in range(today.year - target_date.year):
        webdriver.ActionChains(driver).move_to_element(last_year_button).click(last_year_button).perform()
        time.sleep(2)

    if target_date.month > today.month:
        for _ in range(target_date.month - today.month):
            next_month_button.click()
            time.sleep(2)
    else:
        for _ in range(today.month - target_date.month):
            last_month_button.click()
            time.sleep(2)

    # now find day
    first_day_id = 100
    first_day_text = ''

    # first skip the days from the last month
    while first_day_text != '1':
        first_day_id += 1
        first_day_css = f"td#calendar-date-{first_day_id}"
        first_day_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, first_day_css))
        )
        first_day_text = first_day_div.text
        # print(first_day_id, first_day_css, first_day_text)

    # now move to target day
    target_day = str(target_date.day)
    while first_day_text != target_day:
        first_day_id += 1
        first_day_css = f"td#calendar-date-{first_day_id}"
        first_day_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, first_day_css))
        )
        first_day_text = first_day_div.text
        # print(first_day_id, first_day_css, first_day_text)

    first_day_div.click()
    time.sleep(2)


In [8]:
def get_soup(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

#     for iframe in soup.find_all(['iframe', 'frame']):
#         iframe.extract()
        
    return soup

In [9]:
def extract_table_data(table):
    df = pd.read_html(table.prettify())[0][1:]
    df.columns = ['item', 'quantity', 'units', 'pct_of_target']
    data = df['pct_of_target'].values.tolist()
    cols = df['item'].values.tolist()
    
    return cols, data

In [10]:
def get_diary_details(driver):
    pass

In [11]:
def get_data(driver, date_str):
    soup = get_soup(driver)
    time.sleep(5)
    tables = soup.find_all('table')
    print(len(tables), 'tables found')
    
    diary = pd.read_html(tables[0].prettify())[0]
    if diary.shape[1] != 8:
        print('first table not the right one. Sleep then try again to load tables')
        driver.refresh()
        time.sleep(15)
        tables = soup.find_all('table')
        diary = pd.read_html(tables[0].prettify())[0]
        
        if diary.shape[1] != 8:
            print('could not load diary')
            print(diary)
            return None, None, None
    
    diary.columns = ['time', 'tbd', 'item', 'quantity', 'unit', 'calories', 'tbd2', 'tbd3']
    diary = diary[~diary['time'].str.contains('kcal', na=False)]
    diary = diary.drop(['time', 'tbd', 'tbd2', 'tbd3'], axis=1)
    weight = diary.loc[diary.item=='Weight', 'quantity'].values[0]
    diary['date'] = date_str
    diary = diary[diary.item != 'Weight']
    exercise_calories = diary.loc[diary.unit == 'minutes', 'calories'].sum()
    diary = diary[diary.unit != 'minutes']
    
    # second table gives energy info including total calories for the day
    energy = pd.read_html(tables[1].prettify())[0][1:]
    total_calories = energy.iloc[0,1]
    daily_data = [date_str, weight, total_calories, exercise_calories]
    daily_columns = ['date', 'weight', 'total_calories', 'excercise_calories']
    
    # third table gives carbohydrate information
    carbs_cols, carbs_data = extract_table_data(tables[2])
    daily_data += carbs_data
    daily_columns += carbs_cols
    
    # fourth table gives fats information
    fats_cols, fats_data = extract_table_data(tables[3])
    daily_data += fats_data
    daily_columns += fats_cols
    
    # fifth table gives protein information
    protein_cols, protein_data = extract_table_data(tables[4])
    daily_data += protein_data
    daily_columns += protein_cols
    
    # 6th table gives vitamins
    vitamin_cols, vitamin_data = extract_table_data(tables[5])
    daily_data += vitamin_data
    daily_columns += vitamin_cols
    
    # 7th table gives minerals
    mineral_cols, mineral_data = extract_table_data(tables[6])
    daily_data += mineral_data
    daily_columns += mineral_cols
    
    return daily_columns, daily_data, diary[['date', 'item', 'quantity', 'unit', 'calories']].values.tolist()

So far so good! Have all the daily data I need except for the diary item details. That requires right clicking on all the elements.

In [17]:
def tomorrow(driver):
    next_day_icon_xpath = '//div[contains(@class, "calendar-sidebar")]//div[@class="diary-date-nav"]//i[2]'
    next_day_icon = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_day_icon_xpath)))
    next_day_icon.click()

In [18]:
FOOD_DIARY_XPATH = "//table[@class='crono-table']//td[@class='diary-time']/parent::tr"
view_edit_xpath = "//*[contains(text(), 'View/Edit')]"

In [19]:
def in_viewport(driver, element):
    viewport_x = driver.execute_script("return window.scrollX")
    viewport_y = driver.execute_script("return window.scrollY")
    viewport_height = driver.execute_script("return window.innerHeight")
    viewport_width = driver.execute_script("return window.innerWidth")
    window_height = driver.execute_script("return window.outerHeight")
    window_width = driver.execute_script("return window.outerWidth")

    loc = element.location
    if loc['x'] < viewport_x or loc['x'] > viewport_x + viewport_width:
        print(f'element is outside the x coordinates {loc["x"]}')
        print(f'viewport x from {viewport_x} to {viewport_x + viewport_width}')
        return False
    if loc['y'] < viewport_y or loc['y'] > viewport_y + viewport_height:
        print(f'element is outside the y coordinates {loc["y"]}')
        print(f'viewport y from {viewport_y} to {viewport_y + viewport_height}')
        return False
    return True

In [20]:
def goto_element(driver, element):
    wait = WebDriverWait(driver, 20)
    
    location = element.location
    driver.execute_script("window.scrollTo({}, {});".format(location['x'], location['y']))
    
    # right click element
    ActionChains(driver).context_click(element).perform()
    
    # wait for the view/edit link to be available
    elem = wait.until(
        EC.presence_of_element_located((By.XPATH, view_edit_xpath))
    )
    
    # click the view/edit button to move to the details page for the food element
    elem.click()

In [22]:
def food_details(driver, date):
    wait = WebDriverWait(driver, 20)
    diary = []
    time.sleep(4)
    diary_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, FOOD_DIARY_XPATH)))
    diary_elements = [wait.until(EC.element_to_be_clickable(e)) for e in diary_elements]
    
    columns = []
    values = []
    
    for element in diary_elements:
        data = element.text.split('\n')
        
        if 'Weight' not in data and 'minutes' not in data:
            item_name = data[1]
            vals = [date, item_name]
            cols = ['date', 'item']
            
            goto_element(driver, element)
            time.sleep(2)
            
            tables = pd.read_html(driver.page_source)
            
            time.sleep(4)
            print(len(tables), 'tables found on details page')
            
            # first table is copy of diary from previous page
            # second table is energy data
            my_table = tables[1][1:]
            display(my_table[:1])
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()
            
            # third table is carbohydrates
            my_table = tables[2][1:]
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()
            
            # fourth table is fats
            my_table = tables[3][1:]
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()
            
            # fifth table is protein
            my_table = tables[4][1:]
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()
            
            # sixth table is vitamins
            my_table = tables[5][1:]
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()
            
            # seventh table is minerals
            my_table = tables[6][1:]
            cols += my_table.iloc[:,0].values.tolist()
            vals += my_table.iloc[:,3].values.tolist()

            columns.append(cols)
            values.append(vals)
            
            driver.back()
            time.sleep(2)
            
#     result = pd.DataFrame(values)
#     result.columns = columns[0]
    
    return columns[0], values
    

In [23]:
def scrape_cronometer(start_date, end_date):
    driver = get_driver(URL)
    goto_day(driver, start_date)
    
    start = datetime.strptime(START_DATE, '%Y-%m-%d')
    end = datetime.strptime(END_DATE, '%Y-%m-%d')
    num_days = (end - start).days + 1
    print(f'scraping {num_days} days of data from {START_DATE} to {END_DATE}')
    
    diary_cols = ['date', 'item', 'quantity', 'unit', 'calories']
    
    data = []
    diary = []
    details = []
    
    for day_delta in range(num_days):
        day = start + timedelta(days=day_delta)
        day_str = day.strftime('%Y-%m-%d')
        print(day_str)
        
        daily_columns, daily_data, daily_diary = get_data(driver, day_str)
        data.append(daily_data)
        diary.extend(daily_diary)
        
        # create dfs and save to csv each day in case of problems
        data_df = pd.DataFrame(data)
        data_df.columns = daily_columns
        data_df.to_csv('daily_data.csv', index=None)
        diary_df = pd.DataFrame(diary)
        diary_df.columns=diary_cols
        diary_df.to_csv('food_diary.csv', index=None)
        
        # right click the food elements to get the details
        daily_columns, daily_details = food_details(driver, day_str)
        details.extend(daily_details)
        details_df = pd.DataFrame(details)
        details_df.columns = daily_columns
        details_df.to_csv('food_details.csv', index=None)
        
        tomorrow(driver)
        time.sleep(5)
        
        

In [1]:
# testing
scrape_cronometer(START_DATE, END_DATE)

Review the csv files...

In [25]:
daily = pd.read_csv('daily_data.csv')
daily

,date,weight,total_calories,excercise_calories,Carbs,Fiber,Starch,Sugars,Net Carbs,Fat,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Sodium,Zinc
0,2021-04-01,212.2,1941.8,-84.87,133%,144%,N/T,N/T,131%,41%,...,177%,379%,271%,173%,254%,131%,208%,91%,174%,266%


In [26]:
pd.read_csv('food_diary.csv')

,date,item,quantity,unit,calories
0,2021-04-01,Decaf,2.00,full recipe,75.28
1,2021-04-01,lemon water,2.00,full recipe,10.56
2,2021-04-01,Banana Protein Surprise,1.25,full recipe,411.86
3,2021-04-01,"Carrots, Cooked From Fresh",2.50,"medium - 6"" to 7"" long",52.50
4,2021-04-01,"Okra, Cooked from Fresh",1.00,"cup, sliced",35.20
5,2021-04-01,"Seeds, Flaxseed",1.00,"tbsp, chopped",37.38
6,2021-04-01,"Onion, White, Yellow or Red, Cooked",1.00,"medium - 2 1/2"" diameter",41.36
7,2021-04-01,"Celery, Cooked",2.00,"small - stalk - 5"" long",5.40
8,2021-04-01,"Potatoes, Russet, Flesh and Skin, Baked",3.00,"potato small (1-3/4"" to 2-1/2"" dia.)",393.30
9,2021-04-01,"Ginger Root, Raw",1.00,"tbsp, sliced",4.80


It has the right diary, just wrong details.

In [27]:
pd.read_csv('food_details.csv')

,date,item,Energy,Alcohol,Caffeine,Water,Carbs,Fiber,Starch,Sugars,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Sodium,Zinc
0,2021-04-01,Decaf,3%,N/T,N/T,19%,5%,5%,N/T,N/T,...,3%,9%,3%,7%,4%,3%,9%,2%,5%,1%
1,2021-04-01,lemon water,0%,N/T,N/T,52%,1%,0%,N/T,N/T,...,6%,22%,0%,5%,0%,1%,1%,0%,5%,2%
2,2021-04-01,Banana Protein Surprise,18%,N/T,N/T,3%,36%,21%,N/T,N/T,...,51%,168%,65%,69%,78%,8%,53%,20%,19%,7%
3,2021-04-01,"Carrots, Cooked From Fresh",2%,N/T,N/T,4%,5%,15%,N/T,N/T,...,5%,3%,6%,4%,10%,6%,10%,2%,6%,3%
4,2021-04-01,"Okra, Cooked from Fresh",2%,N/T,N/T,4%,3%,13%,N/T,N/T,...,12%,15%,6%,14%,20%,7%,6%,1%,1%,6%
5,2021-04-01,"Seeds, Flaxseed",2%,N/T,N/T,0%,1%,6%,N/T,N/T,...,2%,9%,5%,7%,8%,6%,2%,3%,0%,3%
6,2021-04-01,"Onion, White, Yellow or Red, Cooked",2%,N/T,N/T,2%,4%,4%,N/T,N/T,...,2%,7%,3%,2%,6%,5%,5%,1%,0%,2%
7,2021-04-01,"Celery, Cooked",0%,N/T,N/T,1%,0%,2%,N/T,N/T,...,1%,1%,2%,1%,1%,1%,3%,1%,2%,0%
8,2021-04-01,"Potatoes, Russet, Flesh and Skin, Baked",17%,N/T,N/T,8%,35%,32%,N/T,N/T,...,7%,49%,55%,30%,41%,42%,67%,4%,4%,13%
9,2021-04-01,"Ginger Root, Raw",0%,N/T,N/T,0%,0%,0%,N/T,N/T,...,0%,2%,0%,1%,1%,0%,1%,0%,0%,0%
